In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/ktcl/")

KTCL = []

def scrape_page():
    # Locate elements  (container)
    routes = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routes:
        try:
            # Extract route name and link from the route element
            route_name_element = route.find_element(By.XPATH, ".//div[@class='route_details']")
            route_link_element = route.find_element(By.XPATH, ".//a")

            route_name = route_name_element.text
            route_link = route_link_element.get_attribute('href')

            # Append extracted data to list
            KTCL.append({
                'route_name': route_name,
                'route_link': route_link
            })

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 5):
    scrape_page()
    if page_number < 4:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Print the scraped data
for entry in KTCL:
    print(entry)

# Convert to DataFrame and save to CSV
df = pd.DataFrame(KTCL)
df.to_csv("ktcl_details.csv", index=False)

# Close the driver
driver.quit()

df.head()  # Display the first few rows of the DataFrame


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
{'route_name': 'Pune to Goa\nFrom INR 900', 'route_link': 'https://www.redbus.in/bus-tickets/pune-to-goa'}
{'route_name': 'Goa to Pune\nFrom INR 800', 'route_link': 'https://www.redbus.in/bus-tickets/goa-to-pune'}
{'route_name': 'Mumbai to Goa\nFrom INR 800', 'route_link': 'https://www.redbus.in/bus-tickets/mumbai-to-goa'}
{'route_name': 'Goa to Mumbai\nFrom INR 900', 'route_link': 'https://www.redbus.in/bus-tickets/goa-to-mumbai'}
{'route_name': 'Pandharpur to Goa\nFrom INR 735', 'route_link': 'https://www.redbus.in/bus-tickets/pandharpur-to-goa'}
{'route_name': 'Bangalore to Goa\nFrom INR 900', 'route_link': 'https://www.redbus.in/bus-tickets/bangalore-to-goa'}
{'route_name': 'Goa to Pandharpur\nFrom INR 735', 'route_link': 'https://www.redbus.in/bus-tickets/goa-to-pandharpur'}
{'route_name': 'Belagavi to Goa\nFrom INR 170', 'rout

,route_name,route_link
0,Pune to Goa\nFrom INR 900,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune\nFrom INR 800,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa\nFrom INR 800,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Goa to Mumbai\nFrom INR 900,https://www.redbus.in/bus-tickets/goa-to-mumbai
4,Pandharpur to Goa\nFrom INR 735,https://www.redbus.in/bus-tickets/pandharpur-t...


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# Initialize the webdriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)

# Define the CSV file path and read URLs
csv_file_path = r"C:\Users\sonur\OneDrive\Documents\vs code\Details\ktcl_details.csv"
try:
    url_df = pd.read_csv(csv_file_path, on_bad_lines='skip')  # Skip bad lines to avoid errors
    urls = url_df['route_link'].tolist()  # Column name in the CSV is 'route_link'
    print(f"URLs found: {len(urls)}")
except Exception as e:
    print(f"Error reading CSV file: {e}")
    urls = []

# Define a list to store all bus details data
bus_details = []

# Function to scrape bus details from the URL
def scrape_bus_details(url):
    print(f"Scraping data from: {url}")
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    try:
        bus_containers = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".clearfix.bus-item-details")))
        for container in bus_containers:
            try:
                # Extract bus details
                bus_name = container.find_element(By.CSS_SELECTOR, '.travels.lh-24.f-bold.d-color').text.strip() or 'N/A'
                bus_type = container.find_element(By.CSS_SELECTOR, '.bus-type.f-12.m-top-16.l-color.evBus').text.strip() or 'N/A'
                departing_time = container.find_element(By.CSS_SELECTOR, '.dp-time.f-19.d-color.f-bold').text.strip() or 'N/A'
                duration = container.find_element(By.CSS_SELECTOR, '.dur.l-color.lh-24').text.strip() or 'N/A'
                reaching_time = container.find_element(By.CSS_SELECTOR, '.bp-time.f-19.d-color.disp-Inline').text.strip() or 'N/A'
                price = container.find_element(By.CSS_SELECTOR, '.fare.d-block span.f-19.f-bold').text.strip() or 'N/A'
                
                # Attempt to find the star rating element
                try:
                    star_rating = container.find_element(By.CSS_SELECTOR, '.rating-sec .rating span').text.strip() or 'N/A'
                except NoSuchElementException:
                    star_rating = 'N/A'
                
                seat_availability = container.find_element(By.CSS_SELECTOR, '.seat-left').get_attribute('innerText').strip() or 'N/A'
               
                bus_details.append({
                    "bus_name": bus_name,
                    "bus_type": bus_type,
                    "departure_time": departing_time,
                    "duration": duration,
                    "arrival_time": reaching_time,
                    "price": price,
                    "star_rating": star_rating,
                    "seat_availability": seat_availability
                })
            except NoSuchElementException as e:
                print(f"Element not found: {e}")
            except Exception as e:
                print(f"An unexpected error occurred while processing a bus container: {e}")
    except TimeoutException:
        print(f"Timed out waiting for bus containers on page: {url}")
    except Exception as e:
        print(f"An error occurred while scraping {url}: {e}")

# Iterate through each URL and scrape data
for url in urls:
    scrape_bus_details(url)

# Convert to DataFrame and save to CSV
df = pd.DataFrame(bus_details)
df.to_csv("ktcl_alldetails.csv", index=False)

# Print the scraped bus details
print(df.head() if not df.empty else "No data to display")

# Close the driver
driver.quit()


URLs found: 40
Scraping data from: https://www.redbus.in/bus-tickets/pune-to-goa
Scraping data from: https://www.redbus.in/bus-tickets/goa-to-pune
Timed out waiting for bus containers on page: https://www.redbus.in/bus-tickets/goa-to-pune
Scraping data from: https://www.redbus.in/bus-tickets/mumbai-to-goa
Timed out waiting for bus containers on page: https://www.redbus.in/bus-tickets/mumbai-to-goa
Scraping data from: https://www.redbus.in/bus-tickets/goa-to-mumbai
Timed out waiting for bus containers on page: https://www.redbus.in/bus-tickets/goa-to-mumbai
Scraping data from: https://www.redbus.in/bus-tickets/pandharpur-to-goa
Timed out waiting for bus containers on page: https://www.redbus.in/bus-tickets/pandharpur-to-goa
Scraping data from: https://www.redbus.in/bus-tickets/bangalore-to-goa
Timed out waiting for bus containers on page: https://www.redbus.in/bus-tickets/bangalore-to-goa
Scraping data from: https://www.redbus.in/bus-tickets/goa-to-pandharpur
Timed out waiting for bus c